# Open Delft3D NetCDF output file
Abondoned in favor of hvPlot and PyVista

#### Enter manually
* Outputstep duration

In [ ]:
%matplotlib widget
!jupyter nbextension enable --py widgetsnbextension
%load_ext autoreload
import numpy as np
import pandas as pd
import xarray as xr
import numpy.ma as ma
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import BoundaryNorm
from matplotlib.colors import LightSource
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

import cmocean.cm as cmo
from JulesD3D.utils import plotCrossSection, niceCDF, getPlotKeyword, checkMinMax, quickDF
from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox
import ipywidgets as widgets
from seaborn import heatmap
colormap = cmo.deep

In [ ]:
# sediments = [('MediumSand', 0), ('FineSand', 1), ('CoarseSilt', 2), ('MediumSilt', 3)] # Check manually in .mdf file
# ncfilename = '/Users/julesblom/Dropbox/TU/Master/Thesis/Results/trim-GUIM.nc'

In [3]:
# ncfilename = '/Users/julesblom/Dropbox/TU/Master/Thesis/Results/trim-60km_300m_W60Channel.nc'
ncfilename = '/Users/julesblom/Dropbox/TU/Master/Thesis/Results/6_2575Run1.nc'

In [4]:
trim = xr.open_dataset(ncfilename) # Open with xarray
trim = niceCDF(trim, sediments) # Make vector sums

[dropdownOptions_horizontal, dropdownOptions_underlayer, dropdownOptions_vertical] = getPlotKeyword(trim)
# [print(trim[keyword].attrs) for keyword in trim.data_vars] #keyword, 
# print(trim.data_vars)

In [ ]:
trim.grid

In [5]:
## Get some key variables
X = trim.variables['XZ'] # Face X-coordinate of grid points
Y = trim.variables['YZ'] # Face Y-coordinate of grid points
DPS = trim.variables['DPS'] # Bottom depth
sediments = [str(sediment.rstrip()) for sediment in nc.NAMCON.values]
xDim, yDim = [trim.dims['M'], trim.dims['N']]
nr_of_sigma_layers, nr_of_sigma_bounds = [trim.dims['SIG_LYR'], trim.dims['SIG_INTF']]
nr_of_sediment_classes = trim.dims['LSED']
total_outputsteps = trim.dims['time']
nr_of_underlayers, nr_of_underlayersP = [trim.dims['nlyr'], trim.dims['nlyrp1']]
output_duration = 6# minutes  #!!!!!!!!!!!!!!!!!!! WATCH THIS!!!!!!!!!!!!!!!!!!!!!!!
print("Nr of outputsteps", total_outputsteps)
print('Total run time hours:', ((total_outputsteps-1) * output_duration)/60, 'hours')
print('Final output time', str(trim.time[-1].values)[11:19])

Nr of outputsteps 91
Total run time hours: 9.0 hours
Final output time 09:00:00


### Define selectors for (global) plot parameters

In [6]:
outputstep = widgets.IntSlider(min=0, max=(trim.time.size - 1), step=1, description='outputstep')
sigma = widgets.IntSlider(min=0, max=(trim.KMAXOUT_RESTR.shape[0]-1), step=1, description='sigma')
x = widgets.IntSlider(min=0, max=X.shape[0], step=1, description='m')
y = widgets.IntSlider(min=0, max=X.shape[1], step=1, description='n')
underlayer = widgets.IntSlider(min=0, max=nr_of_underlayers, step=1, description='Underlayer')
# list(enumerate(['MediumSand', 'FineSand', 'CoarseSilt', 'MediumSilt']))
ui = widgets.HBox([outputstep, sigma, y, x, underlayer])

def printSliderValues(outputstep, sigma, y, x, underlayer):
    print((outputstep, sigma, y, x, underlayer))

out = widgets.interactive_output(printSliderValues, {'outputstep': outputstep, 'sigma': sigma, 'x': x, 'y': y, 'underlayer': underlayer})

sedimentClass = widgets.RadioButtons(
    options=sediments, # Check manually in .mdf file
    description='Sediment:',
    disabled=False
)

In [ ]:
plotKeyword = widgets.Dropdown(
    options=dropdownOptions_horizontal,
    description='Plot:',
    disabled=False,
)

vertPlotKeyword = widgets.Dropdown(
    options=dropdownOptions_vertical,
    description='Plot vertical:',
    disabled=False,
)

underlayerPlotKeyword = widgets.Dropdown(
    options=dropdownOptions_underlayer,
    description='Plot underlayer:',
    disabled=False,
)

## Velocity map plot

In [1]:
@interact_manual
def VelocityMap(outputstep=outputstep, sigma=sigma):
    plt.close("all")    
    velocity = trim['velocity'].isel(time=outputstep, KMAXOUT_RESTR=sigma).plot()

NameError: name 'interact_manual' is not defined

## Vertical velocity plot

In [ ]:
@interact_manual
def VelocityHorizontal(outputstep=outputstep, M=x):
    plt.close("all")
    velocity = trim ['velocity'].isel(time=outputstep, M=M)

#     velo_title = 'Velocity vector at MC={} at {} hours'.format(M+1, str(trim.time[0].values)[11:19])#(outputstep * output_duration)/60)
    velocity.plot(yincrease=False)
#     velo_fig, velo_ax = plt.subplots()
#     velo_im = velo_ax.imshow(velocity, interpolation='bilinear', cmap=cm.inferno,
#                    origin='upper',
#                    vmax=abs(velocity).max(), vmin=0) 
#     velo_ax.set_title(velo_title)
#     velo_fig.colorbar(velo_im, ax=velo_ax, label=trim.U1.units)
    plt.show()

## Easy Density Map

In [ ]:
@interact_manual
def DensityMap(outputstep=outputstep, sigma=sigma):
    plt.show('all')
    selected_densi = trim.RHO.isel(time=outputstep, KMAXOUT_RESTR=sigma)
    
#     selected_densi.plot(yincrease=False)
#     print('min value', abs(selected_densi).min(), 'max value', abs(selected_densi).max(), )

    densi_fig, densi_ax = plt.subplots()
    densi_title = 'Density map at sigma={} at {:.2f} hours'.format(sigma, (outputstep * output_duration)/60)
    
    densi_im = densi_ax.imshow(selected_densi, cmap=cm.viridis,
                   origin='lower', interpolation='bilinear',
                    vmin=abs(selected_densi).min(), vmax=abs(selected_densi).max() )
    densi_ax.set_title(densi_title)
    densi_fig.colorbar(densi_im, ax=densi_ax, label=trim.RHO.units)
    plt.show()

# Plot horizontal Section

In [ ]:
@interact_manual
def plotMapDropdown(keyword=plotKeyword, outputstep=outputstep, sigma=sigma, sedimentNr=sedimentClass):
    property = trim[keyword]
    propertyName = property.attrs['long_name']

    title = 'Horizontal section of {} at {:.2f} hours'.format(propertyName, (outputstep * output_duration)/60)
    values = property.data
    print("keyword", keyword, values.shape)
    print("outputstep:", outputstep, ", sigma:", sigma)

    if values.shape == (total_outputsteps, nr_of_sigma_layers, xDim, yDim) or values.shape == (total_outputsteps, nr_of_sigma_layers+1, xDim, yDim):
        title += ' at sigma layer {}'.format(sigma)
        plotSection = property[outputstep][sigma,:,:]
    elif values.shape == (total_outputsteps, nr_of_sediment_classes, nr_of_sigma_layers+1, xDim, yDim):
        plotSection = property[outputstep][sedimentNr][sigma+1][:,:]
        title += ' ' + str(sedimentClass.options[sedimentNr][0])        
    elif values.shape == (total_outputsteps, 2, nr_of_sigma_layers+1, xDim, yDim): # For RTUR: Turbulent quantiy per sigma layer
        title += ' at sigma layer {}'.format(sigma)
        plotSection = property[outputstep][1][sigma+1][:,:]
    elif values.shape == (total_outputsteps, nr_of_sediment_classes, xDim, yDim):
        plotSection = property[outputstep][sedimentNr][:,:]
        title += ' ' + str(sedimentClass.options[sedimentNr][0])        
    elif values.shape == (1, nr_of_sediment_classes, xDim, yDim):
        plotSection = property[0][sedimentNr][:,:]
        title += ' ' + str(sedimentClass.options[sedimentNr][0])        
    elif values.shape == (total_outputsteps, xDim, yDim):
        plotSection = property[outputstep][:,:]
    elif values.shape == (xDim, yDim):
        plotSection = property[:,:]
    else:
        print("This property has non-plottable dimensions")
        return
        
    mask0 = np.equal(0, plotSection)
    mask999 = np.equal(-999, plotSection)
    maskSmall = np.equal(10000000000.0, plotSection)
    maskKK = np.equal(-1e+10, plotSection)
    mask = mask0 | mask999 | maskSmall | maskKK
    plotSection = ma.array(plotSection, mask = mask)
#     Jules.quickDF(np.rot90(mask[1:-1,1:-1], k=1))
    
    plotCrossSection(X, Y, plotSection, title, property.units, 'x [m]','y [m]', minValue=abs(plotSection).min(), maxValue=abs(plotSection).max()) # x, vmin=-)Dim x yDim
    

# Plot underlayers

In [ ]:
@interact_manual
def plotUnderlayerDropdown(keyword=underlayerPlotKeyword, outputstep=outputstep, underlayer=underlayer, sedimentNr=sedimentClass):
    property = trim[keyword]
    values = property.data
    propertyName = property.attrs['long_name']
    unit = property.units

    title = 'Underlayer of {} at t={:01.2f} hr'.format(propertyName, (outputstep * output_duration)/60)  
    
    if values.shape == (total_outputsteps, nr_of_sediment_classes, nr_of_underlayers, xDim, yDim):
        plotSection = values[outputstep][sedimentNr][underlayer][:,:]
        title += ' ' + str(sedimentClass.options[sedimentNr][0])    
    elif values.shape == (total_outputsteps, nr_of_underlayers, xDim, yDim) or values.shape == (total_outputsteps, nr_of_underlayersP, xDim, yDim):
        title += ' ' + str(sedimentClass.options[sedimentNr][0])
        plotSection = values[outputstep][underlayer][:,:]
    
    plotSection = ma.masked_values(plotSection, 0)
    print(propertyName)
    print("keyword", keyword, values.shape)
    print("outputstep:", outputstep, ", underlayer:", underlayer)
#     quickDF(plotSection)
        
    plotCrossSection(X, Y, plotSection, title, unit, 'x [m]','y [m]') # xDim x yDim
    plt.show()

# Vertical Cross-sections

In [ ]:
@interact_manual
def VertSection(keyword=vertPlotKeyword, sedimentNr=sedimentClass, outputstep=outputstep, M=x, N=y):
    property = trim[keyword]
    values = property.data
    propertyName = property.attrs['long_name']
    print(values.shape)
    
    # Plot a Y/Z section
    if M != 0:
        title = "Vertical {} Section at x={} at {:.2f} hour".format(keyword, N, (outputstep*output_duration)/60)

        if values.shape == (total_outputsteps, 2, nr_of_sigma_layers, xDim, yDim) or values.shape == (total_outputsteps, 2, nr_of_sigma_layers+2, xDim, yDim) or values.shape == (total_outputsteps, 2, 77, xDim, yDim) or values.shape == (total_outputsteps, 2, nr_of_sigma_layers+1, xDim, yDim) :
            plotSection = values[outputstep][1][:,M,:]
        elif values.shape == (total_outputsteps, nr_of_sediment_classes, nr_of_sigma_layers, xDim, yDim) or values.shape == (total_outputsteps, 2, nr_of_sigma_layers+2, xDim, yDim) or values.shape == (total_outputsteps, 2, 77, xDim, yDim):
            plotSection = values[outputstep][1][:,M,:]  
            title += ' ' + str(sedimentClass.options[sedimentNr][0])
        elif values.shape == (total_outputsteps, nr_of_sigma_layers-2, xDim, yDim) or values.shape == (total_outputsteps, nr_of_sigma_layers, xDim, yDim) or values.shape == (total_outputsteps, nr_of_sigma_layers+1, xDim, yDim): 
            plotSection = values[outputstep][:,M,:]
        else:
            print(values.shape)
            print("Dimensions are not right for vertical section")
            return
        plt.xlabel('x')
    # Plot a X/Z section        
    elif N != 0:
        title = "Vertical {} Section at y={} at {:.2f} hour".format(keyword, N, (outputstep*output_duration)/60)
     
        if values.shape == (total_outputsteps, 2, nr_of_sigma_layers, xDim, yDim) or values.shape == (total_outputsteps, 2, nr_of_sigma_layers+1, xDim, yDim) or values.shape == (total_outputsteps, 2, nr_of_sigma_layers-3, xDim, yDim):
            plotSection = values[outputstep][sedimentNr][:,:,N]
        elif values.shape == (total_outputsteps, nr_of_sediment_classes, nr_of_sigma_layers, xDim, yDim) or values.shape == (total_outputsteps, 2, nr_of_sigma_layers+2, xDim, yDim) or values.shape == (total_outputsteps, 2, 77, xDim, yDim):
            plotSection = values[outputstep][1][:,M,:]  
            title += ' ' + str(sedimentClass.options[sedimentNr][0])
        elif values.shape == (total_outputsteps, nr_of_sigma_layers, xDim, yDim) or values.shape == (total_outputsteps, nr_of_sigma_layers+1, xDim, yDim):
            plotSection = values[outputstep][:,:,N]
        else:
            print(values.shape)
            print("Dimensions are not right for vertical section")
            return
        plt.xlabel('y')
    else:
        print(values.shape)
        print("Error: Both x and y are 0")
        return
 
    # plt.figure(2)
    minValue, maxValue = checkMinMax(plotSection)  #1025, 1030
    print("Section shape", plotSection.shape)
    print("Min value", minValue, "Max value", maxValue)

    fig_vert, ax_vert = plt.subplots(nrows=1, figsize=(18,12))

    ax_vert.set_title(title)

    levels = MaxNLocator(nbins=15).tick_values(minValue, maxValue)
    colormap = plt.get_cmap('viridis') # different colormap?
    norm = BoundaryNorm(levels, ncolors=colormap.N, clip=True)
    plt.ylim(80,50)
    plt.ylabel('sigma')
    
    unit = trim[keyword].units
    
    mesh = ax_vert.pcolormesh(plotSection, cmap=colormap, norm=norm)
    cbar = fig_vert.colorbar(mesh, ax=ax_vert)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel(unit,rotation=90)

#     quickDF(plotSection)


## Sediment volume fraction difference Underlayers [LYRFRAC]

In [ ]:
sedVolFraction = trim['LYRFRAC']

@interact_manual
def sedVolDiffPlot(outputstep=outputstep, sedimentNr=sedimentClass, underlayer=underlayer):
    sedVolFractionZero = sedVolFraction.values[0][sedimentNr][underlayer]
#     quickDF(sedVolFractionZero)
    difference = sedVolFraction.values[outputstep][sedimentNr][underlayer] - sedVolFractionZero
    sedVolDiffTitle = "Sediment volume fraction DIFFERENCE of {sedclass} at t={outputstep} hrs and underlayer nr={underlayer}".format(sedclass=sedimentClass.options[sedimentNr][0], outputstep=(outputstep * output_duration)/60, underlayer=underlayer)

    plotCrossSection(X, Y, difference, sedVolDiffTitle, "Percentage %", 'X [m]', 'Y [m]')

In [ ]:
# plt.close('all')
# print(sedimentClass.options[sedimentClass.value][0])
# yup = sedVolFraction[outputstep.value][sedimentClass.value][underlayer.value].plot()

## Contour plot

In [ ]:
# hillshade_cmap = cmo.deep #cmo.thermal # 
@interact_manual
def contourDeposits(outputstep=outputstep):
    hs_fig, hs_ax = plt.subplots()

    elevation = DPS[outputstep][1:-1]
    maskElevation = np.equal(0, elevation)
    elevation = ma.array(elevation, mask = maskElevation)

    
    # Compare with depth at timestep 0
    eleDiff = trim.DPS[0][1:-1] - elevation

    titleString = "Contour of deposit at t={:.2f} hrs".format(outputstep*output_duration/60)
    
    levels = [0.05,0.1,0.2,0.3]
    norm = cm.colors.Normalize(vmax=levels[-1], vmin=levels[0]) # abs(eleDiff).max(), vmin=-abs(eleDiff).max())
    
    pos = hs_ax.imshow(eleDiff, origin='lower', cmap=colormap, norm=norm)
    hs_ax.contour(eleDiff, levels, colors='k', origin='lower')
    hs_ax.set_title(titleString)
    hs_fig.colorbar(pos, ax=hs_ax)

#     Jules.quickDF(eleDiff)

## Heatmap of deposit height

In [ ]:
print("outputstep", outputstep.value)
elevation = DPS[outputstep.value]
# maskElevation = np.equal(0, elevation)
# elevation = ma.array(elevation, mask = maskElevation)    

eleDiff = trim.DPS0 - elevation

f, ax_heatmap = plt.subplots(figsize=(9, 6))

heatmap(eleDiff, cmap=cmo.delta, ax=ax_heatmap, center=0, robust=True)#, annot=True) 

In [ ]:
# quickDF(trim.DPS[0].values)

In [ ]:
# Jules.quickDF(trim.DPS[outputstep.value].values)

## Deposit height plot

In [ ]:
@interact
def contourDeposits(outputstep=outputstep, M=x):
    eleDiff = trim.DPS[outputstep].isel(M=M) - trim.DPS[0].isel(M=M)
#     print(eleDiff)
    plt.close('all')
    eleDiff.plot()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10), nrows=2,ncols=1,gridspec_kw={'height_ratios': [20,1.5]},constrained_layout=True)

pcm = axs[0].pcolormesh(X[1:-1],Y[1:-1],eleDiff.values[1:-1],cmap='viridis')
pcm.set_edgecolor('face')
cbar=fig.colorbar(pcm,cax=axs[1], extend='both', orientation='horizontal')
cbar.set_label('eleDiff [$\mu$g m$^{-3}]$')

## 3D Deposits surface

In [ ]:
@interact_manual
def depositsSurface(outputstep=outputstep):
    fig_3d = plt.figure(figsize=(15,10))
    ax_3d = fig_3d.gca(projection='3d')
    
    elevation = DPS[outputstep][1:-1]
    
    maskElevation = np.equal(0, elevation)
#     maskNaN = ma.masked_invalid(elevation)
#     mask = maskElevation & maskNaN
    elevation = ma.array(elevation, mask = maskElevation)

    # maskElevation = np.equal(0, elevation)
    # elevation = ma.arr|ay(elevation, mask = maskElevation)

    eleDiff = DPS[0][1:-1] - elevation
    surface = ax_3d.plot_surface(X[1:-1], Y[1:-1], eleDiff, cmap=colormap, label='Surface plot of deposits')
    fig_3d.colorbar(surface, aspect=7)
    
    maxDepositHeight = np.amax(eleDiff)
    print("Max deposit height is", maxDepositHeight.data)
    dir(maxDepositHeight)

    ax_3d.view_init(70, 25)
    ax_3d.set_xlabel('X [m]')
    ax_3d.set_ylabel('Y [m]')
#     ax_3d.set_zlim(0,maxDepositHeight.data)
    ax_3d.set_zlabel('Deposit height [m]')

## New Bathymetry

In [ ]:
depthMap = DPS[outputstep.value]

fig_3d = plt.figure(figsize=(15,10))
ax_3d = fig_3d.gca(projection='3d')

surface = ax_3d.plot_surface(X[1:-1], Y[1:-1], depthMap[1:-1], cmap=colormap, label='Bathymetry after xx')
fig_3d.colorbar(surface, shrink=0.5, aspect=7)
ax_3d.view_init(20, 40)
ax_3d.set_xlabel('X [m]')
ax_3d.set_ylabel('Y [m]')
ax_3d.set_zlabel('Depth [m]')

ax_3d.set_zlim(600,0)

# Actual Depth!

In [27]:
trim['actual_depth'] = trim.SIG_LYR @ trim.DPS

/Users/julesblom/.local/share/virtualenvs/ThesisPython-OwMgd5UG/lib/python3.7/site-packages/xarray/core/computation.py:1188: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  return result.transpose(*[d for d in all_dims if d in result.dims])


In [28]:
z_section = trim['actual_depth'].isel(time=20, M=30)

In [35]:
z_section.values

array([[  -3.5900002,   -3.8049476,   -4.0211844, ...,  -25.130001 ,
         -25.130001 ,  -25.130001 ],
       [ -10.18     ,  -10.789517 ,  -11.402691 , ...,  -71.26     ,
         -71.26     ,  -71.26     ],
       [ -15.93     ,  -16.883793 ,  -17.843306 , ..., -111.51     ,
        -111.51     , -111.51     ],
       ...,
       [ -99.92     , -105.9026   , -111.921104 , ..., -699.44     ,
        -699.44     , -699.44     ],
       [ -99.955    , -105.9397   , -111.960304 , ..., -699.685    ,
        -699.685    , -699.685    ],
       [ -99.985    , -105.9715   , -111.993904 , ..., -699.89496  ,
        -699.89496  , -699.89496  ]], dtype=float32)

In [31]:
gridstep = 300 # [m]

# Grid dimensions
length = 60000 # y [m]
width = 18000 # x [m]

cool_xList = [i for i in range(int(gridstep/2), 202 * gridstep, gridstep)] # CENTERs But really y dimensions
XZ, YZ = np.meshgrid(cool_xList, trim.SIG_LYR.values)

In [1]:
density_x_section = trim.RHO.isel(time=20, M=30)

NameError: name 'trim' is not defined

In [33]:
print(density_x_section.shape)
# print(z_section.T.shape)
print(hmm_section.shape)
print(XZ.shape)

(80, 202)
(80, 202)
(80, 202)


In [34]:
fig_vert, ax_vert = plt.subplots(nrows=1, figsize=(18,12))

ax_vert.set_title('Horizontal section')

mesh = ax_vert.pcolormesh(XZ, hmm_section, density_x_section, vmin=1025, vmax=1025.5) #, cmap=colormap, norm=norm)
cbar = fig_vert.colorbar(mesh, ax=ax_vert)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('kg/m3',rotation=90)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'kg/m3')

In [ ]:
plt.close('all')